In [18]:
import numpy as np
import pandas as pand
import tensorflow as tf
import os, random
import pickle
from glob import glob
from time import localtime, strftime
from collections import OrderedDict
from tensorflow.examples.tutorials.mnist import input_data

In [19]:
data_folder='data/cifar10/'
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['data/cifar10/test_batch']

print("done")

done


In [6]:
# NETWORK TOPOLOGIES
list_class=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
n_input    = 32
n_TotalInput=n_input*n_input
n_channel=3

n_NumDense=n_TotalInput//32 #3 pooling layer
n_classes  = len(list_class)  
n_layer=3

# INPUTS AND OUTPUTS
x = tf.placeholder("float", [100, n_TotalInput*n_channel])
y = tf.placeholder("float", [100, n_classes])
    
# NETWORK PARAMETERS
stddev = 0.1
weights = {
    'c1': tf.Variable(tf.random_normal([3, 3, n_channel, 32], stddev=stddev)),
    'c2': tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=stddev)),
    'c3': tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=stddev)),
    'd1': tf.Variable(tf.random_normal([16*128, n_classes], stddev=stddev))
}
biases = {
    'c1': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3': tf.Variable(tf.random_normal([128], stddev=stddev)),
    'd1': tf.Variable(tf.random_normal([n_classes], stddev=stddev))
}
print ("NETWORK READY")

NETWORK READY


In [7]:
# MODEL
def VGGNet(img_width, img_height, img_channel, _x, _w, _b, scope='Simple_VGG'):
    
    _names = OrderedDict()
    
    
    # RESHAPE
    _x_r = tf.reshape(_x, shape=[-1,img_width,img_height, img_channel])
    
    with tf.variable_scope(scope):
        # CONVOLUTION
        
        _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1, 1, 1, 1], padding='SAME')
        # ADD BIAS
        _conv1 = tf.nn.bias_add(_conv1, _b['c1'])
        # RELU
        _conv1 = tf.nn.relu(_conv1)
        _names['Conv3x3_1'] = _conv1
        # MAX-POOL
        _pool  = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _names['pool1'] = _pool
        
        conv2 = tf.nn.conv2d(_pool, _w['c2'], strides=[1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.bias_add(conv2, _b['c2'])
        conv2 = tf.nn.relu(conv2)
        _names['Conv3x3_2'] = conv2
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _names['pool2'] = pool2
        
        
        conv3 = tf.nn.conv2d(pool2, _w['c3'], strides=[1, 1, 1, 1], padding='SAME')
        conv3 = tf.nn.bias_add(conv3, _b['c3'])
        conv3 = tf.nn.relu(conv3)
        _names['Conv3x3_3'] = conv3
        pool3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        _names['pool3'] = pool3
        
        
        # VECTORIZE
        _dense = tf.reshape(pool3, [-1, _w['d1'].get_shape().as_list()[0]])
        _names['dense'] = _dense
        # DENSE
        _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
        
        _names['logit'] = _logit
            
    return _names



In [8]:
# PREDICTION
cnnout = VGGNet(n_input, n_input, n_channel, x, weights, biases,'Simple_VGG')
print(cnnout)
# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=y, logits=cnnout['logit']))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(cnnout['logit'], 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")


OrderedDict([('Conv3x3_1', <tf.Tensor 'Simple_VGG/Relu:0' shape=(100, 32, 32, 32) dtype=float32>), ('pool1', <tf.Tensor 'Simple_VGG/MaxPool:0' shape=(100, 16, 16, 32) dtype=float32>), ('Conv3x3_2', <tf.Tensor 'Simple_VGG/Relu_1:0' shape=(100, 16, 16, 64) dtype=float32>), ('pool2', <tf.Tensor 'Simple_VGG/MaxPool_1:0' shape=(100, 8, 8, 64) dtype=float32>), ('Conv3x3_3', <tf.Tensor 'Simple_VGG/Relu_2:0' shape=(100, 8, 8, 128) dtype=float32>), ('pool3', <tf.Tensor 'Simple_VGG/MaxPool_2:0' shape=(100, 4, 4, 128) dtype=float32>), ('dense', <tf.Tensor 'Simple_VGG/Reshape:0' shape=(100, 2048) dtype=float32>), ('logit', <tf.Tensor 'Simple_VGG/Add:0' shape=(100, 10) dtype=float32>)])
FUNCTIONS READY


In [9]:
print(y)

Tensor("Placeholder_1:0", shape=(100, 10), dtype=float32)


In [10]:
print(cnnout['dense'])

Tensor("Simple_VGG/Reshape:0", shape=(100, 2048), dtype=float32)


In [11]:
savedir = "nets/cnn_cifar10_vgg/"
saver = tf.train.Saver(max_to_keep=3)
save_step = 4
if not os.path.exists(savedir):
    os.makedirs(savedir)
print ("SAVER READY")

SAVER READY


In [12]:
print(cnnout)

OrderedDict([('Conv3x3_1', <tf.Tensor 'Simple_VGG/Relu:0' shape=(100, 32, 32, 32) dtype=float32>), ('pool1', <tf.Tensor 'Simple_VGG/MaxPool:0' shape=(100, 16, 16, 32) dtype=float32>), ('Conv3x3_2', <tf.Tensor 'Simple_VGG/Relu_1:0' shape=(100, 16, 16, 64) dtype=float32>), ('pool2', <tf.Tensor 'Simple_VGG/MaxPool_1:0' shape=(100, 8, 8, 64) dtype=float32>), ('Conv3x3_3', <tf.Tensor 'Simple_VGG/Relu_2:0' shape=(100, 8, 8, 128) dtype=float32>), ('pool3', <tf.Tensor 'Simple_VGG/MaxPool_2:0' shape=(100, 4, 4, 128) dtype=float32>), ('dense', <tf.Tensor 'Simple_VGG/Reshape:0' shape=(100, 2048) dtype=float32>), ('logit', <tf.Tensor 'Simple_VGG/Add:0' shape=(100, 10) dtype=float32>)])


In [13]:
def load_npz(file):
  
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [14]:
## Make batch_iterator for npz data
def make_batch_index(_data, batch_size=128, allow_small_batch=True):
    num_images = len(_data[b'data'])
    start_idx = list(range(0, num_images, batch_size))
    end_idx = list(range(batch_size, num_images + 1, batch_size))
    if allow_small_batch and end_idx[-1] < num_images :
        start_idx.append(end_idx[-1])
        end_idx.append(num_images)
    return zip(start_idx, end_idx)

In [15]:
def one_hot_encode(x, numclass):
    return np.eye(numclass)[x]

In [16]:
training_epochs = 1000
batch_size      = 100
disp_each       = 10
disp_batch      =100
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

In [17]:
load_testdata=load_npz(test_file[0])

FileNotFoundError: [Errno 2] No such file or directory: 'data/cifar10/test_batch'

In [ ]:
for epoch in range(training_epochs):
    
    sample_list_new = random.sample(train_file, len(train_file))
    nNumBatch=0;
    AvgBatchCost=0
    
    for index, file_name in enumerate(sample_list_new):
        file_name=data_folder+file_name
        load_data=load_npz(file_name)
        batch_index=make_batch_index(load_data,batch_size, True)
        
        
        for start, end in batch_index:
            
            batch_xs = load_data[b'data'][start:end]
            batch_ys = one_hot_encode(load_data[b'labels'][start:end],n_classes)
            #print(batch_ys)
            #print(batch_xs.shape)
            feeds = {x: batch_xs, y: batch_ys}
            sess.run(optm, feed_dict=feeds)
            nNumBatch+=1
            tmp_cost, _ = sess.run([cost, optm], feed_dict=feeds)
            AvgBatchCost += tmp_cost
            
        if nNumBatch%disp_batch==0:
                train_acc = sess.run(accr, feed_dict=feeds)
                print('\t[%d nNumBatch] train_cost = %g, acc=%g'%(nNumBatch, AvgBatchCost/nNumBatch, train_acc))
           
    if epoch%disp_each==0:
        num_batch=0
        test_acc=0
        avg_acc=0
        load_testdata=load_npz(test_file[0])
        testdatalabel=one_hot_encode(load_testdata[b'labels'],n_classes)
        test_batch_index=make_batch_index(load_data,batch_size, True)
        for start, end in test_batch_index:
            num_batch+=1;
            batch_xs = load_testdata[b'data'][start:end]
            batch_ys = one_hot_encode(load_testdata[b'labels'][start:end],n_classes)
            test_acc = sess.run(accr, feed_dict={x:batch_xs,y:batch_ys})
            avg_acc+=test_acc
        print('[%d epoch]test acc=%g'%(epoch, avg_acc/num_batch))
        
    

In [ ]:
test_batch_index

In [ ]:
for start, end in test_batch_index:
    print(start, end)